In [1]:
import torch

import torch.nn    as nn
import torch.optim as optim

from torch.utils.data       import random_split
from torch_geometric.data   import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn     import GraphConv

In [2]:
n_epochs      = 1000
batch_size    = 128
learning_rate = 0.0001

target = 'D'

input_folder  = 'models'
target_folder = f'{input_folder}/{target}'
model_name    = f'{target_folder}/model.pt'

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [3]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

try:    
    dataset    = torch.load(dataset_name_std)
    labels     = torch.load(labels_name)
    parameters = torch.load(parameters_name_std)

    # Assigning parameters accordingly
    target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters
    
    # Defining target factor
    target_factor = target_std / scale
except FileNotFoundError:
    dataset = torch.load(dataset_name)
    labels  = torch.load(labels_name)
    
    ### Santadirizing properties

    # Compute means and standard deviations

    target_list = torch.tensor([])
    edge_list   = torch.tensor([])

    for data in dataset:
        target_list = torch.cat((target_list, data.y),         0)
        edge_list   = torch.cat((edge_list,   data.edge_attr), 0)

    scale = 1e0

    target_mean = torch.mean(target_list)
    target_std  = torch.std(target_list)

    edge_mean = torch.mean(edge_list)
    edge_std  = torch.std(edge_list)

    target_factor = target_std / scale
    edge_factor   = edge_std   / scale

    # Update normalized values into the database

    for data in dataset:
        data.y         = (data.y         - target_mean) / target_factor
        data.edge_attr = (data.edge_attr - edge_mean)   / edge_factor

    # Same for the node features

    feat_mean = torch.tensor([])
    feat_std  = torch.tensor([])

    for feat_index in range(dataset[0].num_node_features):
        feat_list = torch.tensor([])

        for data in dataset:
            feat_list = torch.cat((feat_list, data.x[:, feat_index]), 0)

        feat_mean = torch.cat((feat_mean, torch.tensor([torch.mean(feat_list)])), 0)
        feat_std  = torch.cat((feat_std,  torch.tensor([torch.std(feat_list)])),  0)

        for data in dataset:
            data.x[:, feat_index] = (data.x[:, feat_index] - feat_mean[feat_index]) * scale / feat_std[feat_index]
    
    parameters = [target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale]
    
    torch.save(dataset,    dataset_name_std)
    torch.save(parameters, parameters_name_std)

# Generation of Graph Neural Network model

In [ ]:
class diffusion(torch.nn.Module):  # Diffusion
    """Graph generative denoising neural network with edge diffusion.
    """
    
    def __init__(self, features_channels, pdropout):
        """Instantiate constants for the class.
        """
        
        super(diffusion, self).__init__()
        
        self.conv1 = GraphConv(features_channels, 64)
        self.conv2 = GraphConv(64, 64)
        self.conv3 = GraphConv(64, features_channels)
        
        self.pdropout = pdropout
    
    def forward(self, x, edge_index, edge_attr, alpha_t):
        """Diffusion process: applies a Markov chain to the input graph, to diffuse it in T steps.
        """
        
        # Generate random t, epsilon
        
        epsilon = random graph
        # Define sigma parameter
        sigma_t = torch.sqrt(1 - alpha_t**2)
        
        return x, edge_index, edge_attr

In [4]:
class DDPM(torch.nn.Module):  # Denoising
    """Graph generative denoising neural network with edge diffusion.
    """
    
    def __init__(self, features_channels, pdropout):
        """Instantiate constants for the class.
        """
        
        super(DDPM, self).__init__()
        
        self.conv1 = GraphConv(features_channels, 64)
        self.conv2 = GraphConv(64, 64)
        self.conv3 = GraphConv(64, features_channels)
        
        self.pdropout = pdropout
    
    def forward(self, x, edge_index, edge_attr):
        """Denoising process: predicts the original graph from the noised one.
        """
        
        # Denoising process through deep mesh
        x = self.conv1(x, edge_index, edge_attr)
        x = self.conv2(x, edge_index, edge_attr)
        x = self.conv3(x, edge_index, edge_attr)
        return x, edge_index, edge_attr

In [5]:
# torch.manual_seed(12345)

# Define the sizes of the train and test sets
train_size = int(0.9 * len(dataset))
test_size  = len(dataset) - train_size

# Use random_split() to generate train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing  graphs: {len(test_dataset)}')

Number of training graphs: 204
Number of testing  graphs: 23


In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,        shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=len(test_dataset), shuffle=True)

In [7]:
# Define your dataset and data loader
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create the model
model = DDPM(features_channels=dataset[0].num_node_features,
             pdropout=0.4)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Training loop
for epoch in range(n_epochs):
    train_loss = 0
    for data in train_loader:
        optimizer.zero_grad()

        # Forward pass
        out = model(data.x, data.edge_index, data.edge_attr)

        # Calculate the loss
        loss = criterion(out, data.x)

        # Accumulate the training loss
        train_loss += loss.item()

        # Backpropagation and optimization step
        loss.backward()
        optimizer.step()
    
    # Compute the average train loss
    train_loss = train_loss / len(train_loader)
    
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            # Perform a single forward pass
            out_x = model(data.x, data.edge_index, data.edge_attr)

            # Calculate the loss
            loss = criterion(out, data.x)

            # Accumulate the training loss
            test_loss += loss.item()
    
    # Compute the average test loss
    test_loss = test_loss / len(test_loader)
    
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch: 1, Train Loss: 10347380.0000, Test Loss: 6194468.0000
Epoch: 2, Train Loss: 7515795.7500, Test Loss: 4521819.5000
Epoch: 3, Train Loss: 5812716.5000, Test Loss: 3742563.2500
Epoch: 4, Train Loss: 4134142.1250, Test Loss: 3624987.5000
Epoch: 5, Train Loss: 3188670.2500, Test Loss: 3933710.0000
Epoch: 6, Train Loss: 2333513.5000, Test Loss: 4425003.0000
Epoch: 7, Train Loss: 1979343.3750, Test Loss: 4904834.0000
Epoch: 8, Train Loss: 1802134.3750, Test Loss: 5150466.5000
Epoch: 9, Train Loss: 1519395.8750, Test Loss: 5098913.5000
Epoch: 10, Train Loss: 1401079.4375, Test Loss: 4901725.5000
Epoch: 11, Train Loss: 1358810.5938, Test Loss: 4481043.0000
Epoch: 12, Train Loss: 1070170.9375, Test Loss: 3909091.0000
Epoch: 13, Train Loss: 934936.2812, Test Loss: 3336404.5000
Epoch: 14, Train Loss: 832634.3125, Test Loss: 2781900.2500
Epoch: 15, Train Loss: 744776.2188, Test Loss: 2278548.7500
Epoch: 16, Train Loss: 739187.5781, Test Loss: 1844393.3750
Epoch: 17, Train Loss: 578549.1875, 

Epoch: 143, Train Loss: 14665.8042, Test Loss: 37237.3672
Epoch: 144, Train Loss: 15557.9761, Test Loss: 36983.5547
Epoch: 145, Train Loss: 16041.5518, Test Loss: 36472.1523
Epoch: 146, Train Loss: 15424.4644, Test Loss: 35834.0898
Epoch: 147, Train Loss: 13697.2568, Test Loss: 35087.1992
Epoch: 148, Train Loss: 14228.5654, Test Loss: 34495.5352
Epoch: 149, Train Loss: 13074.4077, Test Loss: 34110.6250
Epoch: 150, Train Loss: 14353.8286, Test Loss: 33882.7383
Epoch: 151, Train Loss: 14137.3423, Test Loss: 33809.7578
Epoch: 152, Train Loss: 15180.3550, Test Loss: 33797.1250
Epoch: 153, Train Loss: 13191.0513, Test Loss: 33627.9258
Epoch: 154, Train Loss: 14356.9785, Test Loss: 33528.6953
Epoch: 155, Train Loss: 14366.8325, Test Loss: 33263.0938
Epoch: 156, Train Loss: 12965.3374, Test Loss: 32773.1289
Epoch: 157, Train Loss: 14821.7407, Test Loss: 32379.7129
Epoch: 158, Train Loss: 13592.6338, Test Loss: 31861.0684
Epoch: 159, Train Loss: 14102.2642, Test Loss: 31233.1250
Epoch: 160, Tr

Epoch: 286, Train Loss: 5624.4607, Test Loss: 12606.2168
Epoch: 287, Train Loss: 5532.1125, Test Loss: 12528.4678
Epoch: 288, Train Loss: 5613.3853, Test Loss: 12504.1143
Epoch: 289, Train Loss: 5640.6748, Test Loss: 12397.6279
Epoch: 290, Train Loss: 5262.3425, Test Loss: 12216.8945
Epoch: 291, Train Loss: 5910.5524, Test Loss: 12091.8271
Epoch: 292, Train Loss: 5433.1592, Test Loss: 11946.8428
Epoch: 293, Train Loss: 5513.3962, Test Loss: 11882.5898
Epoch: 294, Train Loss: 5100.6589, Test Loss: 11765.3604
Epoch: 295, Train Loss: 5176.5527, Test Loss: 11697.0693
Epoch: 296, Train Loss: 5376.4724, Test Loss: 11694.0791
Epoch: 297, Train Loss: 5170.6787, Test Loss: 11626.7246
Epoch: 298, Train Loss: 5299.3689, Test Loss: 11565.5137
Epoch: 299, Train Loss: 5465.7830, Test Loss: 11429.9717
Epoch: 300, Train Loss: 4865.5807, Test Loss: 11280.8701
Epoch: 301, Train Loss: 5000.4966, Test Loss: 11171.8750
Epoch: 302, Train Loss: 5161.4041, Test Loss: 11100.3877
Epoch: 303, Train Loss: 5320.32

Epoch: 432, Train Loss: 2487.8512, Test Loss: 5422.0254
Epoch: 433, Train Loss: 2503.2804, Test Loss: 5367.6035
Epoch: 434, Train Loss: 2663.1072, Test Loss: 5325.8999
Epoch: 435, Train Loss: 2623.8007, Test Loss: 5266.3062
Epoch: 436, Train Loss: 2400.4097, Test Loss: 5199.9053
Epoch: 437, Train Loss: 2404.8075, Test Loss: 5147.8911
Epoch: 438, Train Loss: 2431.1703, Test Loss: 5117.1846
Epoch: 439, Train Loss: 2586.3077, Test Loss: 5096.0898
Epoch: 440, Train Loss: 2538.1165, Test Loss: 5098.8037
Epoch: 441, Train Loss: 2497.2756, Test Loss: 5083.8521
Epoch: 442, Train Loss: 2346.6804, Test Loss: 5072.4385
Epoch: 443, Train Loss: 2238.2199, Test Loss: 5068.0103
Epoch: 444, Train Loss: 2558.0328, Test Loss: 5068.1235
Epoch: 445, Train Loss: 2503.2020, Test Loss: 5045.7539
Epoch: 446, Train Loss: 2296.5625, Test Loss: 5017.0195
Epoch: 447, Train Loss: 2324.8244, Test Loss: 4995.6714
Epoch: 448, Train Loss: 2497.3394, Test Loss: 4988.0835
Epoch: 449, Train Loss: 2383.5073, Test Loss: 49

Epoch: 579, Train Loss: 1506.0051, Test Loss: 2976.0596
Epoch: 580, Train Loss: 1415.3485, Test Loss: 2957.6133
Epoch: 581, Train Loss: 1465.5713, Test Loss: 2947.6399
Epoch: 582, Train Loss: 1491.4724, Test Loss: 2949.7993
Epoch: 583, Train Loss: 1434.4515, Test Loss: 2951.1780
Epoch: 584, Train Loss: 1452.7490, Test Loss: 2939.5085
Epoch: 585, Train Loss: 1487.0314, Test Loss: 2912.7354
Epoch: 586, Train Loss: 1374.8000, Test Loss: 2886.0190
Epoch: 587, Train Loss: 1518.7537, Test Loss: 2876.2378
Epoch: 588, Train Loss: 1487.3509, Test Loss: 2870.0525
Epoch: 589, Train Loss: 1450.7213, Test Loss: 2879.7861
Epoch: 590, Train Loss: 1395.3582, Test Loss: 2866.0750
Epoch: 591, Train Loss: 1446.2165, Test Loss: 2841.2844
Epoch: 592, Train Loss: 1429.6632, Test Loss: 2823.0696
Epoch: 593, Train Loss: 1387.5179, Test Loss: 2816.9351
Epoch: 594, Train Loss: 1443.6310, Test Loss: 2821.9211
Epoch: 595, Train Loss: 1400.1238, Test Loss: 2831.7161
Epoch: 596, Train Loss: 1370.1614, Test Loss: 28

Epoch: 726, Train Loss: 978.3255, Test Loss: 1914.7642
Epoch: 727, Train Loss: 998.6888, Test Loss: 1892.5447
Epoch: 728, Train Loss: 961.8082, Test Loss: 1896.9071
Epoch: 729, Train Loss: 971.5745, Test Loss: 1917.9449
Epoch: 730, Train Loss: 980.0600, Test Loss: 1920.4180
Epoch: 731, Train Loss: 967.2055, Test Loss: 1895.3136
Epoch: 732, Train Loss: 977.5005, Test Loss: 1894.3568
Epoch: 733, Train Loss: 1007.2693, Test Loss: 1903.4102
Epoch: 734, Train Loss: 956.2083, Test Loss: 1904.0927
Epoch: 735, Train Loss: 1024.3697, Test Loss: 1894.6455
Epoch: 736, Train Loss: 957.0377, Test Loss: 1877.7943
Epoch: 737, Train Loss: 958.0572, Test Loss: 1871.5897
Epoch: 738, Train Loss: 991.2341, Test Loss: 1865.2242
Epoch: 739, Train Loss: 963.3066, Test Loss: 1868.6370
Epoch: 740, Train Loss: 966.6685, Test Loss: 1862.4282
Epoch: 741, Train Loss: 952.8315, Test Loss: 1856.3542
Epoch: 742, Train Loss: 922.0622, Test Loss: 1851.7928
Epoch: 743, Train Loss: 968.7503, Test Loss: 1845.4056
Epoch: 7

Epoch: 875, Train Loss: 722.7395, Test Loss: 1379.2299
Epoch: 876, Train Loss: 753.9919, Test Loss: 1384.6583
Epoch: 877, Train Loss: 743.3348, Test Loss: 1356.7975
Epoch: 878, Train Loss: 726.9947, Test Loss: 1338.5338
Epoch: 879, Train Loss: 712.6666, Test Loss: 1325.5828
Epoch: 880, Train Loss: 697.6516, Test Loss: 1337.0350
Epoch: 881, Train Loss: 718.6653, Test Loss: 1346.8932
Epoch: 882, Train Loss: 709.8927, Test Loss: 1348.7992
Epoch: 883, Train Loss: 714.2797, Test Loss: 1353.4418
Epoch: 884, Train Loss: 703.5414, Test Loss: 1344.8022
Epoch: 885, Train Loss: 717.1600, Test Loss: 1330.5546
Epoch: 886, Train Loss: 704.0805, Test Loss: 1322.5448
Epoch: 887, Train Loss: 702.8260, Test Loss: 1317.6239
Epoch: 888, Train Loss: 729.3225, Test Loss: 1311.9430
Epoch: 889, Train Loss: 740.7264, Test Loss: 1314.4785
Epoch: 890, Train Loss: 744.5918, Test Loss: 1325.1644
Epoch: 891, Train Loss: 709.9610, Test Loss: 1322.8639
Epoch: 892, Train Loss: 709.9890, Test Loss: 1304.5620
Epoch: 893

In [12]:
'''# Generate a denoised graph

n_atoms = 302
n_edges = 6480

# Define the random graph data
x = torch.randn(n_atoms, 5)
edge_index = torch.randn(2, n_edges)
edge_attr = torch.randn(n_edges, 1)'''

cloned_graph = dataset[0].clone()

n_atoms = cloned_graph.num_nodes
n_edges = cloned_graph.num_edges

x          = cloned_graph.x
edge_index = cloned_graph.edge_index
edge_attr  = cloned_graph.edge_attr

# Create a new Data object with the generated graph data
noisy_graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Set the model in evaluation mode
model.eval()

# Generate a new graph
with torch.no_grad():
    generated_graph = model(noisy_graph.x, noisy_graph.edge_index, noisy_graph.edge_attr)

In [13]:
cloned_graph.x

tensor([[-0.5290, -0.5382,  0.2129,  0.2277, -1.6697],
        [-0.5290, -0.5382,  0.2129,  0.2277, -1.6697],
        [-0.5290, -0.5382,  0.2129,  0.2277, -1.6697],
        ...,
        [ 2.1337,  2.1224,  0.3285,  0.0069, -1.6697],
        [ 2.1337,  2.1224,  0.3285,  0.0069, -1.6697],
        [ 2.1337,  2.1224,  0.3285,  0.0069, -1.6697]])

In [14]:
generated_graph

tensor([[ -0.0679,   0.1566,   0.0869,  -0.0210,   0.0487],
        [ -0.0679,   0.1566,   0.0869,  -0.0210,   0.0487],
        [ -0.0679,   0.1566,   0.0869,  -0.0210,   0.0487],
        ...,
        [ -1.3510,  -5.5763, -11.5359,  -4.2516,  -4.0746],
        [ -1.0962,  -5.8635, -12.1308,  -4.4616,  -4.4294],
        [ -0.6836,  -4.9054, -12.6395,  -3.0544,  -2.7997]])